In [82]:
from os import walk
import xml.etree.ElementTree as ET

In [90]:
path = 'F:\PhD\ABSA\Data\OpeNER\opinion_annotations_en\kaf\hotel'
files = [filename for (dirpath, dirnames, filename) in walk(path)]

In [2]:
file = 'F:\PhD\ABSA\Data\OpeNER\opinion_annotations_en\kaf\hotel\english00001_039d4b7b17acc9ef4243f32de0e3c70e.kaf'

In [3]:
def get_root(file):
    root = ET.parse(file).getroot()
    return root

In [4]:
root = get_root(file)

In [5]:
def get_text(root):
    text = [e.text for c in root.getchildren() for e in c.findall('wf') if c.tag == 'text']
    return text

In [100]:
text = get_text(root)

In [101]:
' '.join(text)

'Comments on my stay at Club Hotel Dolphin Room service needs to be improved and we experienced that some of the Linen provided are damaged . Condition of the room is very poor and also it was not cleaned properly . Attending for issues are ok .'

In [35]:
def get_term(root):
    """return map between tid and wid"""
    terms = [e for c in root.getchildren() for e in c.findall('term') if c.tag == 'terms']
    tw_map = {x.findall('span')[0].findall('target')[0].attrib['id']: x.attrib['tid'] for x in terms}
    return tw_map

In [36]:
tw_map = get_term(root)    

In [93]:
def get_opinions(root):
    """Return target, expression, polarity"""
    opinions = [e for c in root.getchildren() for e in c.findall('opinion') if c.tag == 'opinions']
    triples = []
    for opinion in opinions:
        targets = [o.findall('span')[0].findall('target') for o in opinion.getchildren() if o.tag == 'opinion_target'][0]
        t_id = [t.attrib['id'] for t in targets]
        exps = [e.findall('span')[0].findall('target') for e in opinion.getchildren() if e.tag == 'opinion_expression'][0]
        e_id = [e.attrib['id'] for e in exps]
        polarity = [e.attrib['polarity'] for e in opinion.getchildren() if e.tag == 'opinion_expression'][0]
        triples.append((t_id,e_id,polarity))
    return triples

In [96]:
opinions = get_opinions(root)
opinions

[(['t27', 't28', 't29', 't30'], ['t32', 't33'], 'StrongNegative'),
 (['t27', 't28', 't29', 't30'], ['t37', 't38', 't39', 't40'], 'Negative'),
 (['t42', 't43', 't44'], ['t46'], 'Positive'),
 (['t9', 't10'], ['t11', 't12', 't13', 't14'], 'Negative'),
 (['t19', 't20', 't21', 't22'], ['t25'], 'Negative')]

In [112]:
def id2words(opinions, root):
    text = get_text(root)
    w_opins = []
    for o in opinions:
        target = [w for w in map(lambda x: text[int(x.split('t')[-1])-1],o[0])]
        exp = [w for w in map(lambda x: text[int(x.split('t')[-1])-1],o[1])]
        polarity = o[2]
        w_opins.append([' '.join(target),' '.join(exp),polarity])
    return w_opins

In [113]:
id2words(opinions, root)

[['Condition of the room', 'very poor', 'StrongNegative'],
 ['Condition of the room', 'was not cleaned properly', 'Negative'],
 ['Attending for issues', 'ok', 'Positive'],
 ['Room service', 'needs to be improved', 'Negative'],
 ['some of the Linen', 'damaged', 'Negative']]